# Mass extracting dashboard information

In [ ]:
# importing necessary modules
import numpy as np
import pandas as pd
from pbixray import PBIXRay
from datetime import datetime
import os

In [ ]:
# defining a function to pass a variable name as text to be reused
def get_var_name(var):
    for name, value in globals().items():
        if value is var:
            return name

# defining a function to mass write dataframes to excel
def massWriting_toExcel(filename, dataframes, sheetnames):
    with pd.ExcelWriter(f"{filename}_{str(datetime.now().strftime('%Y-%b-%d.%H.%M'))}.xlsx") as writer:
        for df, sheet in zip(dataframes, sheetnames):
            df.to_excel(writer, sheet_name=sheet, index=False)

# defining a function to extract multiple file paths from a given folder and clean them for variable names uses
def massPrint_fileMapping(path):
    # Iterate over each file in the folder
    for filename in os.listdir(path):
        # Check if the file is a .pbix file
        if filename.endswith('.pbix'):
            # Get the full path of the file
            file_path = os.path.join(path, filename)
            var_name = filename.replace(' ', '_').replace('.pbix','').replace('&','').replace('-','').replace('__','_')
            # print the code-ready variables along with their file path
            print(f"{var_name} = PBIXRay(r'{file_path}')")

In [ ]:
# assigning the folder of interest to a variable
folder_path = r'C:\Users\myname\folder'

# Iterate over each file in the folder
massPrint_fileMapping(folder_path)

This function will output code-ready cleaned file paths in format `variable = PBIXRay(folder_path)` that can be used straight away by copy-paste into the code cell below.

In [ ]:
dashboard1 = PBIXRay(r'C:\Users\myname\dashboard1.pbix')
dashboard2 = PBIXRay(r'C:\Users\myname\dashboard2.pbix')
dashboard3 = PBIXRay(r'C:\Users\myname\dashboard3.pbix')

In [ ]:
# similarly to the individual extraction file, grouping dataframes into a list 
# with sheet names in another list ready for zip later on
pbix_files = [dashboard1, dashboard2, dashboard3]
sheet_names = ['dashboard1', 'dashboard2', 'dashboard3']

In [ ]:
# excel has a limit of 31 characters for its sheet names so need for verification
# based on the result of the loop, correction can be made to the sheet names list above
for i in sheet_names:
    if len(i) > 31:
        print(i, ": ", len(i) - 31, "characters to be removed")

Now that the two lists are ready, the documentation can begin.

Compared to the individual extraction process where all information is extracted at once into one excel file, this process extracts the same information for all dashboards to export into an excel file. So for example, all metadata is extracted at once, or all queries are extracted at once.

The metadata can be extracted altogether into one tab thanks to its simple format.

In [ ]:
# defining a list to hold dataframe information on metadata
global_metadata = []

# looping through the dashboards to extract the metadata
for i in pbix_files:
    model_info = {}
    model_info['name'] = get_var_name(i)
    model_info['Mb_size'] = round(i.size / 1e+6, 2)
    metadata = i.metadata.sort_values(by= 'Name')
    model_info['desktop_version'] = metadata.iloc[0, 1]
    model_info['TimeIntelligence_Enabled'] = metadata.iloc[2, 1]
    model_info['queries'] = metadata.iloc[1, 1]
    global_metadata.append(model_info)

# consolidating the looped extractions into a dataframe that can be exported
global_metadata = pd.DataFrame(global_metadata).sort_values(by= 'name')
global_metadata.to_csv("Metadata.csv", index= False)

The format to follow to mass extract is as follows:
1. defining a dictionary to store the dataframes further created in the process
2. looping through the dashboards with one measure to analyze (example below is looking for DAX measures)
3. stripping the dataframes of metadata to focus on their values of interest
4. exporting to excel

In [ ]:
# Dictionary to store DataFrames for each value in pbix_files
global_measures = {}

for i, pbix_file in enumerate(pbix_files):
    model_measures = pbix_file.dax_measures
    # Create a DataFrame for each value of i and store it in the dictionary
    global_measures[f"{pbix_file}"] = pd.DataFrame(model_measures)

global_measures = list(global_measures.values())

massWriting_toExcel("measures",
                    dataframes= global_measures,
                    sheetnames= sheet_names)